## Initialize

In [ ]:
import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import requests
import json
import base64
from PyPDF2 import PdfFileReader
import fitz

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import (TENANT_SUFFIX, StopExecution, get_base_url,
                               get_root_path, print_response)

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()
root_path = get_root_path()


### Select example which should be used

In [ ]:
example_path = os.path.join('..')
options = next(os.walk(example_path))[1]
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

### Create data.json

In [ ]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        docType = ''
        filename_blank = ''
        created_at_string = ''
        changed_at_string = ''
        author = ''
        encoded_img = ''
        encoded_doc = ''
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            if filename.endswith('pdf'):
                docType = 'PDF'
                filename_blank = filename.replace('.pdf', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')

                # Get file info
                pdf = PdfFileReader(f)
                info = pdf.getDocumentInfo()

                created_at = str(info.creation_date_raw)
                created_at_string = created_at[2:10]
                changed_at = str(info.modification_date_raw)
                changed_at_string = changed_at[2:10]
                print(filename_blank + ' Created at: ' + created_at_string +
                      ' Changed at: ' + changed_at_string)
                if info.author == None:
                    author = 'Not specified'
                else:
                    author = info.author

                # get image
                pdf_file = fitz.open(os.path.join(
                    current_path, filename))
                page = pdf_file[0]
                image_page = page.get_pixmap()
                image_page.shrink(2)
                encoded_string_img = base64.b64encode(image_page.tobytes())
                encoded_img = encoded_string_img.decode('ascii')
                encoded_img = 'data:;base64,' + encoded_img

                
            elif filename.endswith('doc'):
                
                docType = 'DOC'
                filename_blank = filename.replace('.doc', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                  
            elif filename.endswith('docx'):
                docType = 'DOCX'
                filename_blank = filename.replace('.docx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xls'):
                docType = 'XLS'
                filename_blank = filename.replace('.xls', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xlsx'):
                docType = 'XLSX'
                filename_blank = filename.replace('.xlsx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('msg'):
                docType = 'MSG'
                filename_blank = filename.replace('.msg', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {
                        'image': encoded_img,
                        'title': filename_blank,
                        'author': author,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string,
                        'docType': docType
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)
    #save data.json
    with open(f'../{selected.value}/data.json','w') as fjson:        
        json.dump(payload_list, fjson)

else:
    print('No folder selected for files')


### Create CSON (JSON-format) from CDS file (text format) using the CDS compiler

In [ ]:
if selected.value:
    cson_file_direct = os.path.join(root_path, '..', 'examples', 'DocResearch', 'model.json')
    if os.path.exists(cson_file_direct):
        with open(cson_file_direct, encoding='utf-8') as f:
            cson = json.load(f)
    else:
        cds_file = os.path.join(root_path, '..',  'model.cds')
        cson_file = os.path.join(root_path, '..','..','..', 'shared', 'temp', 'DocResearch.model.json')
        command_line_statement = (f'cds compile {cds_file} --to json -o {cson_file}')
        os.system(command_line_statement)
        with open(cson_file, encoding='utf-8') as f:
            cson = json.load(f)
        print(f'The following command line statement was executed to compile the example cds file to cson. Filename is temp{os.sep}DocResearch.model.json):')
        print(command_line_statement)
    data_file = os.path.join(root_path, '..', 'examples', 'DocResearch',selected.value, 'data.json')
else:
    print('No example selected')

## Create Tenant, deploy, load data on SAP Server

In [ ]:
headers = {'Content-Type': 'application/json'}
#Create Tenant
r = requests.post(f'http://vlcspdj7db.devsys.net.sap:8031/v1/tenant/DocResearch{TENANT_SUFFIX}')
print_response(r)
#Deploy 
r = requests.post(f'http://vlcspdj7db.devsys.net.sap:8031/v1/deploy/DocResearch{TENANT_SUFFIX}', json=cson)
print_response(r)
#Load data
if selected.value:
    data_file = os.path.join(root_path, '..', selected.value, 'data.json')
    if os.path.exists(data_file):
        with open(data_file, encoding = 'utf-8') as f:
            data = json.load(f)
        for element in payload_list:
            r = requests.post(f'http://vlcspdj7db.devsys.net.sap:8031/v1/data/DocResearch{TENANT_SUFFIX}', data=element, headers=headers)#json=data)
            if r.status_code == 200:
                print('Data successfully loaded')
            else:
                print(f'Error loading data. HTTP status code {r.status_code}')
                print(r.text)
    else:
        print('No data file exists')


## Delete tenant on SAP server

In [ ]:
if selected.value:
    r = requests.delete(f'http://vlcspdj7db.devsys.net.sap:8031/v1/tenant/DocResearch{TENANT_SUFFIX}')
    print_response(r)
else:
    print('There is no tenant to delete')
